In [1]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.distributions import Normal
from torch.distributions.kl import kl_divergence
import numpy as np
import os
import random

In [2]:
batch_size=512
epochs=500
seed=1
cuda=False and torch.cuda.is_available()
log_interval=1
r_dim=128
z_dim=128
result_path="results_np_z_y_hat_parmu/"
file_name=result_path+"loss"
file=open(file_name,"w")
file.write("Begin\n")
file.close()

In [3]:

torch.manual_seed(seed)
random.seed(seed)
device = torch.device("cpu") #"cuda" if args.cuda else 

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=False,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)


RuntimeError: Dataset not found. You can use download=True to download it

In [4]:
def get_context_idx(N):
    # generate the indeces of the N context points in a flattened image
    idx = random.sample(range(0, 784), N)
    idx = torch.tensor(idx, device=device)
    return idx
def generate_grid(h, w):
    rows = torch.linspace(0, 1, h, device=device)
    cols = torch.linspace(0, 1, w, device=device)
    grid = torch.stack([cols.repeat(h, 1).t().contiguous().view(-1), rows.repeat(w)], dim=1)
    grid = grid.unsqueeze(0)
    return grid
def idx_to_y(idx, data):
    # get the [0;1] pixel intensity at each index
    y = torch.index_select(data, dim=1, index=idx)
    return y
def idx_to_x(idx, batch_size):
    # From flat idx to 2d coordinates of the 28x28 grid. E.g. 35 -> (1, 7)
    # Equivalent to np.unravel_index()
    x = torch.index_select(x_grid, dim=1, index=idx)
    x = x.expand(batch_size, -1, -1)
    return x

In [5]:
class NP(nn.Module):
    def __init__(self, r_dim,z_dim):
        super(NP, self).__init__()
        self.r_dim = r_dim
        self.z_dim = z_dim
    
        self.h_1 = nn.Linear(3, 400)
        self.h_2 = nn.Linear(400, 400)
        self.h_3 = nn.Linear(400, self.r_dim)

        self.r_to_z_mean = nn.Linear(self.r_dim, self.z_dim)
        self.r_to_z_logvar = nn.Linear(self.r_dim, self.z_dim)

        self.g_1 = nn.Linear(self.z_dim + 2, 400)
        self.g_2 = nn.Linear(400, 400)
        self.g_3 = nn.Linear(400, 400)
        self.g_4 = nn.Linear(400, 400)
        self.g_y_mu = nn.Linear(400, 1)
        self.g_y_sigma = nn.Linear(400, 1)
        

    def h(self, x_y):
        x_y = F.relu(self.h_1(x_y))
        x_y = F.relu(self.h_2(x_y))
        x_y = F.relu(self.h_3(x_y))
        return x_y

    def aggregate(self, r):
        return torch.mean(r, dim=1)

    def g(self,z_sample, x_target):
        z_et_x = torch.cat([z_sample, x_target], dim=2)
        input = F.relu(self.g_1(z_et_x))
        input = F.relu(self.g_2(input))
        input = F.relu(self.g_3(input))
        input = F.relu(self.g_4(input))
        y_mu=self.g_y_mu(input)
        y_sigma=sigma = 0.1 + 0.9 * F.softplus(self.g_y_sigma(input))
        return y_mu,y_sigma
    
    
    def xy_to_z_params(self, x, y):
        
        x_y = torch.cat([x, y], dim=2)
        
        r_i = self.h(x_y)
        r = self.aggregate(r_i)
        mu = self.r_to_z_mean(r)
        logvar = self.r_to_z_logvar(r)
        sigma=0.1+0.9*torch.sigmoid(logvar)
        return mu, sigma

    def forward(self, x_context, y_context, x_all=None, y_all=None):
        
        #produire z
        z_context_mu,z_context_sigma = self.xy_to_z_params(x_context, y_context)  # (mu, logvar) of z
        q_context = Normal(z_context_mu, z_context_sigma)
        # reconstruct the whole image including the provided context points
        x_target = x_grid.expand(y_context.shape[0], -1, -1)
        
        if self.training:  # loss function will try to keep z_context close to z_all         
            z_target_mu,z_target_sigma = self.xy_to_z_params(x_context, y_context)
            q_target = Normal(z_target_mu, z_target_sigma) 
            z_sample = q_target.rsample()
            
            z_sample = z_sample.unsqueeze(1).expand(-1, 784, -1)
            
            # Get parameters of output distribution
            y_pred_mu, y_pred_sigma = self.g(z_sample,x_target)
            p_y_pred = Normal(y_pred_mu, y_pred_sigma)

            return p_y_pred, q_target, q_context
        else:  # at test time we don't have the image so we use only the context
            z_sample = q_context.rsample()
            z_sample = z_sample.unsqueeze(1).expand(-1, 784, -1)
            # Predict target points based on context
            y_pred_mu, y_pred_sigma = self.g(z_sample,x_target)
            p_y_pred = Normal(y_pred_mu, y_pred_sigma)
            return p_y_pred,q_context,q_context

In [6]:

def np_loss(p_y_pred, y_target, q_target, q_context):
    
    #return logprob + KLD
    log_likelihood = p_y_pred.log_prob(y_target).mean(dim=0).sum()
    # KL has shape (batch_size, r_dim). Take mean over batch and sum over
    # r_dim (since r_dim is dimension of normal distribution)
    kl = kl_divergence(q_target, q_context).mean(dim=0).sum()
    return -log_likelihood + kl
    
    

In [7]:
model = NP(r_dim,z_dim).to(device)
model.load_state_dict(torch.load(result_path+"model_{}.pt".format(z_dim)))
model.eval()
learning_rate=5e-4
x_grid = generate_grid(28, 28)
os.makedirs(result_path, exist_ok=True)

In [8]:
def train(epoch):
    model.train()
    train_loss = 0
    optimizer = optim.Adam(model.parameters(), lr=learning_rate/np.sqrt(epoch))
    for batch_idx, (y_all, _) in enumerate(train_loader):
        
        batch_size = y_all.shape[0]
        
        y_all = y_all.to(device).view(batch_size, -1, 1)
        
        N = random.randint(1, 784)  # number of context points
        context_idx = get_context_idx(N)
        x_context = idx_to_x(context_idx, batch_size)
        y_context = idx_to_y(context_idx, y_all)
        x_all = x_grid.expand(batch_size, -1, -1)

        optimizer.zero_grad()
        
        p_y_pred, q_target, q_context = model(x_context, y_context, x_all, y_all)
        
        loss = np_loss(p_y_pred, y_all, q_target, q_context)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            file=open(file_name,"a")
            file.write('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\n'.format(
                epoch, batch_idx * len(y_all), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(y_all)))
            file.close()
            
    file=open(file_name,"a")
    file.write('====> Epoch: {} Average loss: {:.4f}\n'.format(
        epoch, train_loss / len(train_loader.dataset)))
    file.close()
    return train_loss / len(train_loader.dataset)

In [27]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (y_all, _) in enumerate(test_loader):
            y_all = y_all.to(device).view(y_all.shape[0], -1, 1)
            batch_size = y_all.shape[0]

            N = 300
            context_idx = get_context_idx(N)
            x_context = idx_to_x(context_idx, batch_size)
            y_context = idx_to_y(context_idx, y_all)
            
            p_y_pred, q_target, q_context = model(x_context, y_context)
        
            
            test_loss += np_loss(p_y_pred, y_all, q_target, q_context).item()

            if i == 0:  # save PNG of reconstructed examples
                plot_Ns = [10, 100, 300, 784]
                num_examples = min(batch_size, 16)
                for N in plot_Ns:
                    recons = []
                    recons1=[]
                    recons2=[]
                    context_idx = get_context_idx(N)
                    x_context = idx_to_x(context_idx, batch_size)
                    y_context = idx_to_y(context_idx, y_all)
                    for d in range(5):
                        p_y_pred, _, _ = model(x_context, y_context)
                        
                        recons.append(p_y_pred.rsample()[:num_examples])
                        recons1.append(p_y_pred.mean[:num_examples])  
                        recons2.append(p_y_pred.stddev[:num_examples])  
                    recons = torch.cat(recons).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    recons1 = torch.cat(recons1).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    recons2 = torch.cat(recons2).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    background = torch.tensor([0., 0., 1.], device=device)
                    background = background.view(1, -1, 1).expand(num_examples, 3, 784).contiguous()
                    context_pixels = y_all[:num_examples].view(num_examples, 1, -1)[:, :, context_idx]
                    context_pixels = context_pixels.expand(num_examples, 3, -1)
                    background[:, :, context_idx] = context_pixels
                    comparison = torch.cat([background.view(-1, 3, 28, 28),
                                            recons,recons1,recons2])
                    save_image(comparison.cpu(),
                               result_path+"ep_" + str(epoch) +
                               "_nps_" + str(N) + ".png", nrow=num_examples)

    test_loss /= len(test_loader.dataset)
    file=open(file_name,"a")
    file.write('====> Test set loss: {:.4f}\n'.format(test_loss))
    file.close()

In [36]:
loss_min=10
for epoch in range(134, epochs + 1):
    loss_epoch=train(epoch)
    if loss_epoch < loss_min:
        torch.save(model.state_dict(), result_path+"model_{}_ep_{}.pt".format(z_dim,epoch))
        loss_min=loss_epoch
    test(epoch)

Train Epoch: 78 [0/60000 (0%)]	Loss: -0.758361
Train Epoch: 78 [512/60000 (1%)]	Loss: -0.769944
Train Epoch: 78 [1024/60000 (2%)]	Loss: -0.783140
Train Epoch: 78 [1536/60000 (3%)]	Loss: -0.735293
Train Epoch: 78 [2048/60000 (3%)]	Loss: -0.762766


KeyboardInterrupt: 

In [37]:
torch.save(model.state_dict(), result_path+"model_{}.pt".format(z_dim))